In [1]:
import tensorflow as tf
import numpy as np

x_data = [[1, 2, 1, 1],
         [2, 1, 3, 2],
         [3, 1, 3, 4],
         [4, 1, 5, 5],
         [1, 7, 5, 5],
         [1, 2, 5, 6],
         [1, 6, 6, 6],
         [1, 7, 7, 7]]

# one-hot encoding
y_data = [[0, 0, 1],
         [0, 0, 1],
         [0, 0, 1],
         [0, 1, 0],
         [0, 1, 0],
         [0, 1, 0],
         [1, 0, 0],
         [1, 0, 0]]

x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

nb_classes = 3

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [2]:
W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

print(variables)

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.19633393, -0.67894936,  0.07726288],
       [-0.72127634, -0.9497583 , -2.4160798 ],
       [ 0.2874941 , -1.1374302 , -0.03768032],
       [ 0.82390803,  1.7027009 , -0.3149968 ]], dtype=float32)>, <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.6632706 , -1.1701741 ,  0.90436906], dtype=float32)>]


In [11]:
sample_db = [[8, 2, 1, 4]]
sample_db = np.asarray(sample_db, dtype=np.float32)
hypothesis = tf.nn.softmax(tf.matmul(sample_db, W) + b)

print(hypothesis) # Softmax를 이용한 one-hot encoding
print(tf.reduce_sum(hypothesis)) # 각 확률치를 더한 값 = 1

tf.Tensor([[2.0018742e-06 9.9999797e-01 5.0492748e-09]], shape=(1, 3), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [3]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[9.6782494e-01 2.3646800e-02 8.5283229e-03]
 [9.9117821e-01 1.7668776e-03 7.0548435e-03]
 [9.9506915e-01 4.2863437e-03 6.4452097e-04]
 [9.9965394e-01 2.4999675e-04 9.6041374e-05]
 [9.9912351e-01 8.7644544e-04 5.2613958e-09]
 [9.9341488e-01 6.5770885e-03 8.0206755e-06]
 [9.9936193e-01 6.3808059e-04 6.6283721e-09]
 [9.9970585e-01 2.9418091e-04 2.8162558e-10]], shape=(8, 3), dtype=float32)


In [4]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(4.6780252, shape=(), dtype=float32)


In [10]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x * x
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [5]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        cost = cost_fn(X, Y)
        grads = tape.gradient(cost, variables)
        return grads
print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 1.4907014 , -0.74382186, -0.7468795 ],
       [ 1.7370448 , -1.2401532 , -0.4968915 ],
       [ 2.7352052 , -1.8642235 , -0.8709817 ],
       [ 2.8598685 , -1.9880865 , -0.87178195]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.74316657, -0.37020802, -0.3729585 ], dtype=float32)>]


In [7]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1) % verbose == 0):
            print('Loss at epoch {}: {}'.format(i+1, cost_fn(X,Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 4.342783451080322
Loss at epoch 100: 0.9157935380935669
Loss at epoch 200: 0.772110641002655
Loss at epoch 300: 0.6907076835632324
Loss at epoch 400: 0.6399660110473633
Loss at epoch 500: 0.6062471270561218
Loss at epoch 600: 0.5819138288497925
Loss at epoch 700: 0.562882125377655
Loss at epoch 800: 0.5470860004425049
Loss at epoch 900: 0.5334579944610596
Loss at epoch 1000: 0.5214025974273682
Loss at epoch 1100: 0.510557234287262
Loss at epoch 1200: 0.5006817579269409
Loss at epoch 1300: 0.49160635471343994
Loss at epoch 1400: 0.48320499062538147
Loss at epoch 1500: 0.4753805100917816
Loss at epoch 1600: 0.4680556356906891
Loss at epoch 1700: 0.4611680507659912
Loss at epoch 1800: 0.4546663463115692
Loss at epoch 1900: 0.44850751757621765
Loss at epoch 2000: 0.4426555633544922


In [8]:
sample_data = [[2, 1, 3, 2]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1))

tf.Tensor([[0.03448453 0.08983088 0.87568456]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [9]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[0.02944318 0.05076008 0.91979676]
 [0.03448451 0.08983088 0.87568456]
 [0.00414131 0.4842729  0.51158583]
 [0.00367719 0.61517006 0.38115278]
 [0.57253414 0.39095262 0.03651316]
 [0.27990058 0.71060854 0.0094909 ]
 [0.6153006  0.3769333  0.00776609]
 [0.668747   0.329473   0.00177998]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 0 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [22]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
        self.b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))
        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            
            return grads

    def fit(self, X, Y, epochs=2000, verbose=100):
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch {}: {}'.format(i+1, self.cost_fn(X, Y).numpy()))
    
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 5.452994346618652
Loss at epoch 100: 0.684638500213623
Loss at epoch 200: 0.5929224491119385
Loss at epoch 300: 0.5313973426818848
Loss at epoch 400: 0.47873741388320923
Loss at epoch 500: 0.429477334022522
Loss at epoch 600: 0.3816629648208618
Loss at epoch 700: 0.33455735445022583
Loss at epoch 800: 0.2886422276496887
Loss at epoch 900: 0.2505078911781311
Loss at epoch 1000: 0.234241783618927
Loss at epoch 1100: 0.22254188358783722
Loss at epoch 1200: 0.2119368016719818
Loss at epoch 1300: 0.20227447152137756
Loss at epoch 1400: 0.1934325248003006
Loss at epoch 1500: 0.18530979752540588
Loss at epoch 1600: 0.17782223224639893
Loss at epoch 1700: 0.17089824378490448
Loss at epoch 1800: 0.16447734832763672
Loss at epoch 1900: 0.1585073322057724
Loss at epoch 2000: 0.15294276177883148
